In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

In [ ]:
!pip install git+git://github.com/stared/livelossplot.git

In [ ]:
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt
import numpy as np
from sklearn.model_selection import train_test_split
from keras.preprocessing.sequence import TimeseriesGenerator
from keras.layers.core import Dense, Activation, Dropout
from keras.layers.recurrent import LSTM, GRU
from keras.models import Sequential
from keras import optimizers
import tensorflow as tf
import csv   
from tqdm import tqdm
import time
import livelossplot
from livelossplot.main_logger import MainLogger
from livelossplot import outputs,PlotLossesKeras

# **fetching data**

In [ ]:
train = pd.read_csv("/kaggle/input/tabular-playground-series-jul-2021/train.csv")
test = pd.read_csv("/kaggle/input/tabular-playground-series-jul-2021/test.csv")
submission=pd.read_csv("/kaggle/input/tabular-playground-series-jul-2021/sample_submission.csv")

In [ ]:
train.head(10)

# **Data Visualization**

In [ ]:
train['date_time']=pd.to_datetime(train['date_time'],infer_datetime_format=True)
s=train.set_index('date_time')[['deg_C','relative_humidity','absolute_humidity','sensor_1','sensor_2','sensor_3','sensor_4','sensor_5','target_carbon_monoxide','target_benzene','target_nitrogen_oxides']].plot(subplots=True,figsize=(20,20),marker='v',markersize=5, markevery=100)

In [ ]:
train.describe()

# **Getting the important data that will be processed**

In [ ]:
labels=train[["target_carbon_monoxide","target_benzene","target_nitrogen_oxides"]]

In [ ]:
X_train=np.array(train.drop(["date_time","target_carbon_monoxide","target_benzene","target_nitrogen_oxides"],axis=1))
y_train=np.array(labels)
X_test=np.array(test.drop(["date_time"],axis=1))


In [ ]:
X_train.shape,y_train.shape,X_test.shape

# **Data Normalization**

In [ ]:
X_train = X_train / X_train .max(axis=0)
X_test = X_test / X_test .max(axis=0)
X_train.shape

# **Data Splitting**

In [ ]:
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, random_state=42, test_size=0.3, shuffle=False)
X_train.shape, X_test.shape, X_val.shape, y_train.shape,  y_val.shape

# **Generating time series to train on it**

In [ ]:
f=np.array([np.ravel(X_train, order='C')]).T
generator = TimeseriesGenerator(X_train, y_train, length=1, batch_size=1)
print('Samples: %d' % len(generator))
# print first four samples
for i in range(4):
	x, y = generator[i]
	print('%s => %s' % (x, y))

# **Model architecture and training**

In [ ]:
# define model
model = Sequential()
model.add(LSTM(128,input_shape = (1, 8),return_sequences=True,activation = "tanh"))
model.add(LSTM(32, activation = "sigmoid",return_sequences=False))
model.add(tf.keras.layers.Dropout(0.3))
model.add(Dense(3,  activation = tf.keras.layers.PReLU()))
model.compile(optimizer='adam', loss=tf.keras.losses.MeanSquaredLogarithmicError())

# fit model
generator_val = TimeseriesGenerator(X_val, y_val, length=1, batch_size=1)
model.fit_generator(generator, epochs=10, validation_data = generator_val, verbose=0,callbacks=[PlotLossesKeras()])

# **Predicting the test data**

In [ ]:
predections=X_test[0]
predections = predections.reshape((1,1,8))
predections = model.predict(predections, verbose=0)
for i in tqdm(range(1,X_test.shape[0])):
    temp=X_test[i].reshape((1,1,8))
    temp = model.predict(temp, verbose=0)
    predections=np.vstack((predections, temp)) 

# **Submitting the predictions**

In [ ]:
submission["target_carbon_monoxide"]=predections[:,0]
submission["target_benzene"]=predections[:,1]
submission["target_nitrogen_oxides"]=predections[:,2]
submission.to_csv("submission.csv", index=False)

In [ ]:
submission.head(10)